#Algoritmo a programar 
##Pasos

1. Los compañeros le proponen a su mejor opcion 

2. si uno se repite elige a su mejor propuesta

3. si uno es rechazado va con su siguiente opcion 

4. si uno con pareja tiene otra propuesta y es mejor se va con ella

5. el rechazado busca en el siguiente en su lista 

6. repetir hasta que todos tengan pareja 

Suponemos que está hecha la matriz n x n sin datos repetidos sobre la misma fila y en la primera columna.

In [ ]:
#Libreria importante para pasar del csv a lista de python.
import pandas as pd

In [ ]:
def toMatrix():
  df = pd.read_csv('preferences.csv', header=None)
  return df.values.tolist()

In [ ]:
preferencias   = toMatrix()
preferencias

[[1, 2, 3, 4], [2, 3, 4, 1], [3, 2, 1, 4], [4, 1, 3, 2]]

In [ ]:
def obtener_rank(preferencias):
    rank = [[None for j in range(len(preferencias))] for i in range(len(preferencias))]

    for i in range(len(preferencias)): 
        for j in range(len(preferencias[i])):
            rank[i][preferencias[i][j]] = j

    return rank


In [ ]:
def fase1(preferencias, rank):
    propuesta = [None for x in range(len(preferencias))]
    primero = [0 for x in range(len(preferencias))]
    ultimo = [len(x) for x in preferencias]
    en_lista = [x for x in range(len(preferencias))]

    while len(en_lista) > 0:
        i = en_lista[0]

        #aqui se modifica la primera posicion si es necesario
        while preferencias[i][primero[i]] == None:
            primero[i] +=1

        primera_opcion = preferencias[i][primero[i]]

        #si la primera opcion no tiene propuestas aun, acepta 
        if propuesta[primera_opcion] == None:
            propuesta[primera_opcion] = i

            match_rank = preferencias[primera_opcion].index(i)

            for x in range(match_rank+1, ultimo[primera_opcion]):
                rechazo = preferencias[primera_opcion][x]
                preferencias[rechazo][rank[rechazo][primera_opcion]] = None

            ultimo[primera_opcion] = match_rank
            del en_lista[0]

            continue
        
        actual_match_ind = rank[primera_opcion][propuesta[primera_opcion]]
        potencial_match_ind = rank[primera_opcion][i]

        if actual_match_ind < potencial_match_ind: #el match actual es preferido, i se rechaza
            preferencias[primera_opcion][potencial_match_ind] = None

            primero[i] += 1 #empieza en la siguiente posicion 

            continue

        else: #acepta la propuesta, el match anterior regresa a la lista de preferencias
            preferencias[primera_opcion][actual_match_ind] = None
            #el match viejo es rechazado por la primera opcion, se actualiza la lista
            primera_opcion_ind = rank[propuesta[primera_opcion]][primera_opcion]
            preferencias[propuesta[primera_opcion]][primera_opcion_ind] = None

            del en_lista[0]
            #agregamos el match viejo  a en_lista
            en_lista.insert(0, propuesta[primera_opcion])

            propuesta[primera_opcion] = i
            ultimo[primera_opcion] = potencial_match_ind

    return primero, ultimo, preferencias


In [ ]:
def limpiar_preferencias(primero, ultimo, preferencias):
    for i in range(len(preferencias)):
        for j in range(len(preferencias[i])):
            if j < primero[i] or j > ultimo[i]:
                preferencias[i][j] = None

    return preferencias

In [ ]:
def buscar_el_segundo(i, primero, ultimo, pref):
    contador = 0
    for j in range(primero[i], ultimo[i]+1):
        if not pref[j]  == None:
            contador += 1           
        elif contador == 0:
            primero[i] += 1
        if contador == 2:
            return pref[j]

    return None

In [ ]:
def buscar_rotacion(i, p, q, primero, ultimo, preferencias):
    segundo_fav = buscar_el_segundo(p[i], primero, ultimo, preferencias[p[i]])
    siguiente_pos = preferencias[segundo_fav][ultimo[segundo_fav]]

    if siguiente_pos in p:
        #se busca la rotacion
        j = p.index(siguiente_pos)
        q[j] = segundo_fav

        return p[j:], q[j:]
    
    q.append(segundo_fav)
    p.append(siguiente_pos)
    return buscar_rotacion(i+1, p, q, primero, ultimo, preferencias)


In [ ]:
def eliminar_rotacion(p, q, primero, ultimo, preferencias, rank):
    for i in range(len(p)):
        #q_i rechaza a p_i entonces p_i propone a q_i+1
        preferencias[p[i]][rank[p[i]][q[i]]] = None

        for j in range(rank[q[i]][p[i-1]]+1, ultimo[q[i]]):
            rechazado = rank[q[i]].index(j) 
            preferencias[rechazado][rank[rechazado][q[i]]] = None

        ultimo[q[i]] = rank[q[i]][p[i-1]]


In [ ]:
def fase2(primero, ultimo, preferencias, rank):
    while True:
        p, q = None, None
        #busca el primero p_0 y obtine la rotacion por la lista de preferencias de p_0
        for i in range(len(preferencias)):
            if ultimo[i] - primero[i] > 0 and buscar_el_segundo(i, primero, ultimo, preferencias[i]) !=None:
                p, q = buscar_rotacion(0, [i], [None], primero, ultimo, preferencias)
                break
        
        if not p and not q:
            return preferencias

        #eliminamos rotacion 
        eliminar_rotacion(p, q, primero, ultimo, preferencias, rank)


In [ ]:
def match_compañeros(preferencias):
    rank = obtener_rank(preferencias)
    primero, ultimo, preferencias = fase1(preferencias, rank)
    fase2(primero, ultimo, preferencias, rank)
    limpiar_preferencias(primero, ultimo, preferencias)

    matches = []
    largo = len(preferencias)
    visitado = set()
    i = 0

    for i in range(len(preferencias)):
        if not i in visitado:
            par = (i, preferencias[i][ultimo[i]])
            visitado.add(ultimo[i])
            matches.append(par)

    return matches


In [ ]:
match_compañeros(preferencias)

IndexError: ignored